## Importing Packages

In [34]:
import pandas as pd
import numpy as np
import tqdm
import pickle
from pprint import pprint
import os

import warnings
warnings.filterwarnings('ignore')

#sklearn
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split

import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis

In [2]:
with open('processed_tweets.pickle', 'rb') as read_file:
    df = pickle.load(read_file)

## Train-Test Split

In [3]:
X_train, X_test = train_test_split(df.tweet, test_size=0.2, random_state=42)
X_train

20180    [congrats, lead, nonprofit, organization, dedi...
17391    [youre, pet, owner, new, york, know, toxic, in...
9976                 [restock, feel, good, brooklynwineco]
6257     [since, percent, excess, heat, retain, earth, ...
6245     [shorebird, watch, big, business, new, jersey,...
                               ...                        
11964    [ooh, yourcbdstorebk, union, berkeley, cbd, st...
21575    [meet, horticultural, therapist, garden, progr...
5390     [alone, together, gowanus, community, group, s...
860      [new, york, city, experience, dangerous, heat,...
15795    [congratulations, raise, money, thank, partici...
Name: tweet, Length: 17672, dtype: object

In [4]:
X_test

19530    [bundle, new, yorkers, go, dip, freeze, every,...
20241    [congratulations, award, grant, historic, hous...
15031                            [refresh, flower, street]
9960     [mask, thermometers, oximeters, near, ave, bea...
735      [deeply, thankful, haul, trash, recycle, treat...
                               ...                        
16606      [get_repost, member, dl, dd, show, chefs, take]
21677    [know, secure, food, box, family, four, please...
7948                                  [missamericanpienyc]
4630     [dont, trash, tree, mulch, drop, bbp, anytime,...
8439     [virtual, learn, great, option, yourchild, lea...
Name: tweet, Length: 4418, dtype: object

In [5]:
train_list_of_lists = list(X_train.values)

## Bigram-Trigram Models

(I did not incorporate bigrams and trigrams into the model yet)

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(train_list_of_lists, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[train_list_of_lists], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [7]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [8]:
data_words_bigrams = make_bigrams(train_list_of_lists)

## Bag of Words

In [11]:
id2word = Dictionary(train_list_of_lists)
corpus = [id2word.doc2bow(text) for text in train_list_of_lists]

In [12]:
sample = corpus[3000]

for i in range(len(sample)):
    print("Word {} (\"{}\") appears {} time(s).".format(sample[i][0], 
                                                     id2word[sample[i][0]], 
                                                     sample[i][1]))

Word 173 ("class") appears 1 time(s).
Word 572 ("get_repost") appears 1 time(s).
Word 660 ("wine") appears 1 time(s).
Word 692 ("january") appears 1 time(s).
Word 715 ("pizza") appears 1 time(s).
Word 732 ("february") appears 1 time(s).
Word 833 ("thursday") appears 2 time(s).
Word 1022 ("month") appears 1 time(s).
Word 1091 ("dd") appears 1 time(s).
Word 3509 ("pair") appears 2 time(s).


## LDA with Bag of Words

In [13]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=4, 
                     random_state=42,
                     chunksize=100,
                     passes=100,
                     update_every=5,
                     alpha='auto',
                     per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.019*"water" + 0.010*"river" + 0.008*"protect" + 0.008*"new" + '
  '0.008*"clean" + 0.006*"state" + 0.006*"hudson" + 0.006*"environmental" + '
  '0.006*"society" + 0.006*"littoral"'),
 (1,
  '0.014*"us" + 0.011*"new" + 0.011*"thank" + 0.010*"join" + 0.009*"today" + '
  '0.009*"get" + 0.008*"day" + 0.008*"make" + 0.008*"help" + 0.008*"learn"'),
 (2,
  '0.025*"park" + 0.016*"brooklyn" + 0.012*"st" + 0.010*"come" + '
  '0.009*"get_repost" + 0.008*"slope" + 0.008*"good" + 0.008*"corner" + '
  '0.007*"amaze" + 0.007*"house"'),
 (3,
  '0.011*"high" + 0.010*"line" + 0.009*"june" + 0.008*"friday" + '
  '0.008*"ticket" + 0.007*"april" + 0.006*"bio" + 0.006*"business" + '
  '0.006*"update" + 0.006*"staff"')]


In [23]:
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.322777 -0.131328       1        1  49.998978
0      0.032648  0.315035       2        1  19.579902
2     -0.174582 -0.178563       3        1  17.910235
3     -0.180843 -0.005145       4        1  12.510885, topic_info=          Term         Freq        Total Category  logprob  loglift
21        park  1158.000000  1158.000000  Default  30.0000  30.0000
568      water   836.000000   836.000000  Default  29.0000  29.0000
416   brooklyn   658.000000   658.000000  Default  28.0000  28.0000
376         us  1537.000000  1537.000000  Default  27.0000  27.0000
225         st   497.000000   497.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
421     greene   127.231518   146.363514   Topic4  -5.3918   1.9385
90          pm   126.124528   211.618379   Topic4  -5.4005   1.5611
21        park   153.162776  1158.315906   Topic4  -5.2063   0.0553
1190      gift   107.759324   241.266850   Topic4  -5.5579   1.2726
97         sit   101.000835   208.589036   Topic4  -5.6227   1.3533

[175 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1926      2  0.996015       act
817       3  0.996386     amaze
580       1  0.421017  american
580       2  0.578006  american
1670      4  0.992406   another
...     ...       ...       ...
1058      4  0.995828   whitman
2264      4  0.995067       win
186       1  0.998525      work
724       1  0.998386      year
27        3  0.995935     youre

[163 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4])

In [12]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=train_list_of_lists, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.28826282195420916


In [24]:
lda_model_bow = gensim.models.LdaMulticore(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=4, 
                                            random_state=42,
                                            chunksize=100,
                                            passes=100,
                                            update_every=5,
                                            alpha='auto',
                                            per_word_topics=True,
                                            workers=2)

for idx, topic in lda_model_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.013*"park" + 0.010*"open" + 0.008*"come" + 0.008*"brooklyn" + 0.008*"st" + 0.007*"get" + 0.006*"get_repost" + 0.005*"slope" + 0.005*"day" + 0.005*"us"
Topic: 1 
Words: 0.007*"grant" + 0.007*"garden" + 0.007*"us" + 0.006*"learn" + 0.006*"receive" + 0.005*"make" + 0.005*"gift" + 0.005*"community" + 0.005*"see" + 0.005*"today"
Topic: 2 
Words: 0.016*"water" + 0.010*"new" + 0.009*"river" + 0.007*"protect" + 0.007*"high" + 0.006*"line" + 0.006*"state" + 0.006*"change" + 0.005*"hudson" + 0.005*"fish"
Topic: 3 
Words: 0.017*"thank" + 0.012*"us" + 0.010*"help" + 0.010*"support" + 0.009*"join" + 0.008*"community" + 0.008*"work" + 0.008*"new" + 0.007*"get" + 0.006*"need"


In [25]:
LDAvis_prepared_2 = gensimvis.prepare(lda_model_bow, corpus, id2word)
LDAvis_prepared_2

/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Use

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.011295 -0.131908       1        1  27.525524
0     -0.192323  0.142689       2        1  27.294450
1     -0.057272 -0.106578       3        1  22.873902
2      0.238300  0.095797       4        1  22.306124, topic_info=         Term         Freq        Total Category  logprob  loglift
445     thank  1160.000000  1160.000000  Default  30.0000  30.0000
568     water   876.000000   876.000000  Default  29.0000  29.0000
150     river   467.000000   467.000000  Default  28.0000  28.0000
225        st   479.000000   479.000000  Default  27.0000  27.0000
169   support   792.000000   792.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
185       use   142.680420   348.755398   Topic4  -5.8554   0.6065
218      take   151.586391   615.794511   Topic4  -5.7949   0.0986
26       york   131.883566   366.270613   Topic4  -5.9341   0.4789
166       one   137.951832   659.963096   Topic4  -5.8891  -0.0650
1163     live   128.265984   409.853580   Topic4  -5.9619   0.3386

[271 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1926      1  0.056775         act
1926      4  0.936783         act
647       1  0.983754  appreciate
872       2  0.195551         art
872       3  0.803008         art
...     ...       ...         ...
101       4  0.488288       years
26        1  0.488710        york
26        3  0.150162        york
26        4  0.360389        york
2256      1  0.990090     yorkers

[374 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3])

In [26]:
for index, score in sorted(lda_model_bow[corpus[3000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_bow.print_topic(index, 4)))


Score: 0.9402359127998352	 
Topic: 0.013*"park" + 0.010*"open" + 0.008*"come" + 0.008*"brooklyn"

Score: 0.020550765097141266	 
Topic: 0.007*"grant" + 0.007*"garden" + 0.007*"us" + 0.006*"learn"

Score: 0.019865620881319046	 
Topic: 0.017*"thank" + 0.012*"us" + 0.010*"help" + 0.010*"support"

Score: 0.019347691908478737	 
Topic: 0.016*"water" + 0.010*"new" + 0.009*"river" + 0.007*"protect"


In [27]:
# Compute Coherence Score
coherence_model_lda_2 = CoherenceModel(model=lda_model_bow, texts=train_list_of_lists, dictionary=id2word, coherence='c_v')
coherence_lda_2 = coherence_model_lda_2.get_coherence()
print('Coherence Score: ', coherence_lda_2)

Coherence Score:  0.28302921569964656


## LDA with TF-IDF

In [29]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.42466773753051157),
 (1, 0.1766020120629213),
 (2, 0.23014024341235387),
 (3, 0.2832750364803111),
 (4, 0.2366724011621648),
 (5, 0.2832750364803111),
 (6, 0.18979599067454203),
 (7, 0.2143401059677086),
 (8, 0.24462258623808159),
 (9, 0.33186344059309086),
 (10, 0.29674778220154957),
 (11, 0.19428450256362492),
 (12, 0.19638562074430463),
 (13, 0.24300529787323882),
 (14, 0.2088991128043566)]


In [30]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                                id2word=id2word,
                                                num_topics=4, 
                                                random_state=42,
                                                chunksize=100,
                                                passes=100,
                                                update_every=5,
                                                alpha='auto',
                                                per_word_topics=True,
                                                workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.006*"thank" + 0.005*"us" + 0.004*"join" + 0.004*"support" + 0.004*"community" + 0.003*"learn" + 0.003*"today" + 0.003*"work" + 0.003*"help" + 0.003*"program"
Topic: 1 Word: 0.003*"order" + 0.003*"new" + 0.002*"delivery" + 0.002*"get" + 0.002*"today" + 0.002*"plastic" + 0.002*"open" + 0.002*"day" + 0.002*"park" + 0.002*"slope"
Topic: 2 Word: 0.005*"st" + 0.004*"corner" + 0.003*"open" + 0.003*"union" + 0.003*"store" + 0.003*"ave" + 0.003*"president" + 0.003*"wine" + 0.002*"happy" + 0.002*"come"
Topic: 3 Word: 0.006*"water" + 0.004*"gowanus" + 0.003*"river" + 0.003*"new" + 0.002*"protect" + 0.002*"climate" + 0.002*"clean" + 0.002*"quality" + 0.002*"drink" + 0.002*"ocean"


In [31]:
LDAvis_prepared_3 = gensimvis.prepare(lda_model_tfidf, corpus_tfidf, id2word)
LDAvis_prepared_3

/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Use

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.128487  0.086884       1        1  39.891917
1      0.044824  0.024272       2        1  22.697430
3     -0.057123 -0.129982       3        1  22.266538
2      0.140786  0.018826       4        1  15.144115, topic_info=            Term        Freq       Total Category  logprob  loglift
568        water  101.000000  101.000000  Default  30.0000  30.0000
225           st   78.000000   78.000000  Default  29.0000  29.0000
466      gowanus   61.000000   61.000000  Default  28.0000  28.0000
259       corner   55.000000   55.000000  Default  27.0000  27.0000
412        union   27.000000   27.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
273        great   16.138464   87.104228   Topic4  -6.2480   0.2017
21          park   16.400163  126.403807   Topic4  -6.2319  -0.1546
572   get_repost   14.650467   59.450204   Topic4  -6.3448   0.4869
1029        love   14.515447   66.201027   Topic4  -6.3540   0.3701
30          good   13.740483   51.187597   Topic4  -6.4089   0.5724

[282 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1926      2  0.304443             act
1926      3  0.695870             act
446       1  0.992940      activities
6221      2  0.964616         alchemy
2408      4  0.906691          annies
...     ...       ...             ...
101       3  0.339390           years
101       4  0.018855           years
995       4  0.895915  yourcbdstorebk
1668      1  0.996568           youth
3626      4  0.961039           zuzus

[457 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3])

In [32]:
for index, score in sorted(lda_model_tfidf[corpus[3000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 4)))


Score: 0.5621809959411621	 
Topic: 0.005*"st" + 0.004*"corner" + 0.003*"open" + 0.003*"union"

Score: 0.39833658933639526	 
Topic: 0.006*"thank" + 0.005*"us" + 0.004*"join" + 0.004*"support"

Score: 0.02010035328567028	 
Topic: 0.003*"order" + 0.003*"new" + 0.002*"delivery" + 0.002*"get"

Score: 0.01938203163444996	 
Topic: 0.006*"water" + 0.004*"gowanus" + 0.003*"river" + 0.003*"new"


In [33]:
# Compute Coherence Score
coherence_model_lda_3 = CoherenceModel(model=lda_model_tfidf, texts=train_list_of_lists, dictionary=id2word, coherence='c_v')
coherence_lda_3 = coherence_model_lda_3.get_coherence()
print('Coherence Score: ', coherence_lda_3)

Coherence Score:  0.2654761237827161


## Hyperparameter Tuning

In [35]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=42,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=train_list_of_lists, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [37]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [03:02<?, ?it/s]
Process SpawnPoolWorker-131:
Process SpawnPoolWorker-130:
Process SpawnPoolWorker-133:
Process SpawnPoolWorker-129:
Process SpawnPoolWorker-127:
Process SpawnPoolWorker-132:
Process SpawnPoolWorker-128:
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/examsherpa/opt/anaconda3/envs/nlp-env/lib/python3.8/multiprocessing/process.py", line 108, in run
    sel

KeyboardInterrupt: 